In [1]:
# Código para o desafio do bootcamp de TPD
# Na verdade o primeiro passo seria adicionar o cluster que foi criado

1) Primeiro passo - Seção Spark

In [3]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [4]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("desafio_tpd").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [5]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/Mall_Customers.csv,Mall_Customers.csv,4286
dbfs:/FileStore/tables/adult_data.csv,adult_data.csv,5608318
dbfs:/FileStore/tables/arvoreDecisao-1.ipynb,arvoreDecisao-1.ipynb,40795
dbfs:/FileStore/tables/arvoreDecisao-2.ipynb,arvoreDecisao-2.ipynb,40795
dbfs:/FileStore/tables/arvoreDecisao-3.ipynb,arvoreDecisao-3.ipynb,40795
dbfs:/FileStore/tables/arvoreDecisao.ipynb,arvoreDecisao.ipynb,40795
dbfs:/FileStore/tables/iris.csv,iris.csv,3979


In [6]:
diretorioDatasetDiabets="/FileStore/tables/Mall_Customers.csv"  #diretório que contém o arquivo a ser utilizado

In [7]:
data = spark.read.format("csv").options(header="true", inferschema="true").load(diretorioDatasetDiabets)  #realiza a leitura do dataset

In [8]:
data.show(5,False)  #mostrando as 5 primeiras instâncias do dataset

+----------+------+---+------------------+----------------------+
CustomerID|Genre |Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
1 |Male |19 |15 |39 |
2 |Male |21 |15 |81 |
3 |Female|20 |16 |6 |
4 |Female|23 |16 |77 |
5 |Female|31 |17 |40 |
+----------+------+---+------------------+----------------------+
only showing top 5 rows

In [9]:
data.columns #mostra as colunas do dataset

Out[7]: ['CustomerID', 'Genre', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']

In [10]:
data.printSchema()  #mostra o esquema inferido pelas variáveis

root
-- CustomerID: integer (nullable = true)
-- Genre: string (nullable = true)
-- Age: integer (nullable = true)
-- Annual Income (k$): integer (nullable = true)
-- Spending Score (1-100): integer (nullable = true)

In [11]:
data.count()  #conta a quantidade de registros

Out[9]: 200

In [12]:
#verificando a existência ou não de valores nulos
from pyspark.sql.functions import isnan, when, count, col

data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()

+----------+-----+---+------------------+----------------------+
CustomerID|Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+-----+---+------------------+----------------------+
 0| 0| 0| 0| 0|
+----------+-----+---+------------------+----------------------+

In [13]:
data.describe().show() #mostrando as estatísticas do dataset

+-------+------------------+------+-----------------+------------------+----------------------+
summary| CustomerID| Genre| Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
 count| 200| 200| 200| 200| 200|
 mean| 100.5| null| 38.85| 60.56| 50.2|
 stddev|57.879184513951124| null|13.96900733155888| 26.26472116527124| 25.823521668370173|
 min| 1|Female| 18| 15| 1|
 max| 200| Male| 70| 137| 99|
+-------+------------------+------+-----------------+------------------+----------------------+

Verificando se existem outliers (anomalias) nos dados. Para isso, será utilizado o boxplot.

In [15]:
#selecionando apenas algumas colunas
data_outliers=data.select([c for c in data.columns if c in ['Age','Annual Income (k$)','Spending Score (1-100)']])
data_outliers.show()

+---+------------------+----------------------+
Age|Annual Income (k$)|Spending Score (1-100)|
+---+------------------+----------------------+
 19| 15| 39|
 21| 15| 81|
 20| 16| 6|
 23| 16| 77|
 31| 17| 40|
 22| 17| 76|
 35| 18| 6|
 23| 18| 94|
 64| 19| 3|
 30| 19| 72|
 67| 19| 14|
 35| 19| 99|
 58| 20| 15|
 24| 20| 77|
 37| 20| 13|
 22| 20| 79|
 35| 21| 35|
 20| 21| 66|
 52| 23| 29|
 35| 23| 98|
+---+------------------+----------------------+
only showing top 20 rows

In [16]:
#boxplot
display(data_outliers)

Age,Annual Income (k$),Spending Score (1-100)
19,15,39
21,15,81
20,16,6
23,16,77
31,17,40
22,17,76
35,18,6
23,18,94
64,19,3
30,19,72


In [17]:
data_outliers2=data.select([c for c in data.columns if c in ['Annual Income (k$)']])
data_outliers2.show()

+------------------+
Annual Income (k$)|
+------------------+
 15|
 15|
 16|
 16|
 17|
 17|
 18|
 18|
 19|
 19|
 19|
 19|
 20|
 20|
 20|
 20|
 21|
 21|
 23|
 23|
+------------------+
only showing top 20 rows

In [18]:
display(data_outliers2)

Annual Income (k$)
15
15
16
16
17
17
18
18
19
19


In [19]:
#histograma
display(data_outliers)

Age,Annual Income (k$),Spending Score (1-100)
19,15,39
21,15,81
20,16,6
23,16,77
31,17,40
22,17,76
35,18,6
23,18,94
64,19,3
30,19,72


Conhecendo o dataset

In [21]:
display(data.select(["Genre","Annual Income (k$)", "Age"]))

Genre,Annual Income (k$),Age
Male,15,19
Male,15,21
Female,16,20
Female,16,23
Female,17,31
Female,17,22
Female,18,35
Female,18,23
Male,19,64
Female,19,30


In [22]:
#encontrando a distribuição da idade x sexo
display(data.select(["Age"]))

Age
19
21
20
23
31
22
35
23
64
30


In [23]:
#encontrando a distribuição da idade x sexo
display(data.select(["Age","Genre"]))

Age,Genre
19,Male
21,Male
20,Female
23,Female
31,Female
22,Female
35,Female
23,Female
64,Male
30,Female


A variável "Spending Score (1-100)" ou pontuação de consumo, corresponde à nota atribuída pelo shopping aos consumidores. Quanto mais próxima de 100 indica que o consumidor é mais "lucrativo" para o shopping.

In [25]:
#encontrando as notas atribuidas por gênero 
display(data.select(["Genre", "Annual Income (k$)", "Spending Score (1-100)"]))

Genre,Annual Income (k$),Spending Score (1-100)
Male,15,39
Male,15,81
Female,16,6
Female,16,77
Female,17,40
Female,17,76
Female,18,6
Female,18,94
Male,19,3
Female,19,72


Analisando a correlação entre Idade e Nota Atribuída

In [27]:
#Analisando as correlações
from pyspark.mllib.stat import Statistics

In [28]:
rdd_1=data.select("Spending Score (1-100)").rdd.flatMap(lambda x:x)  #utilizada para transforma das colunas do dataframe em colunas para serem empregadas na análise de correlaçoes
rdd_2=data.select("Age").rdd.flatMap(lambda x:x)

In [29]:
Statistics.corr(rdd_1,rdd_2,method="pearson")  #definindo a coeficiente de correlação de pearson 

Out[23]: -0.32722684603908925

In [30]:
data.stat.corr("Age","Annual Income (k$)",method="pearson")

Out[24]: -0.01239804273606026

In [31]:
data.stat.corr("Age","Spending Score (1-100)",method="pearson")

Out[25]: -0.32722684603909025

Analisando a correlação por gênero

In [33]:
#Selecionando os homens
homens=data[data.Genre=='Male']
homens.show()

+----------+-----+---+------------------+----------------------+
CustomerID|Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+-----+---+------------------+----------------------+
 1| Male| 19| 15| 39|
 2| Male| 21| 15| 81|
 9| Male| 64| 19| 3|
 11| Male| 67| 19| 14|
 15| Male| 37| 20| 13|
 16| Male| 22| 20| 79|
 18| Male| 20| 21| 66|
 19| Male| 52| 23| 29|
 21| Male| 35| 24| 35|
 22| Male| 25| 24| 73|
 24| Male| 31| 25| 73|
 26| Male| 29| 28| 82|
 28| Male| 35| 28| 61|
 31| Male| 60| 30| 4|
 33| Male| 53| 33| 4|
 34| Male| 18| 33| 92|
 42| Male| 24| 38| 92|
 43| Male| 48| 39| 36|
 52| Male| 33| 42| 60|
 54| Male| 59| 43| 60|
+----------+-----+---+------------------+----------------------+
only showing top 20 rows

In [34]:
#selecionando mulheres
mulheres=data[data.Genre=='Female']
mulheres.show()

+----------+------+---+------------------+----------------------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
 3|Female| 20| 16| 6|
 4|Female| 23| 16| 77|
 5|Female| 31| 17| 40|
 6|Female| 22| 17| 76|
 7|Female| 35| 18| 6|
 8|Female| 23| 18| 94|
 10|Female| 30| 19| 72|
 12|Female| 35| 19| 99|
 13|Female| 58| 20| 15|
 14|Female| 24| 20| 77|
 17|Female| 35| 21| 35|
 20|Female| 35| 23| 98|
 23|Female| 46| 25| 5|
 25|Female| 54| 28| 14|
 27|Female| 45| 28| 32|
 29|Female| 40| 29| 31|
 30|Female| 23| 29| 87|
 32|Female| 21| 30| 73|
 35|Female| 49| 33| 14|
 36|Female| 21| 33| 81|
+----------+------+---+------------------+----------------------+
only showing top 20 rows

In [35]:
#correlação entre Idade e o Salário Anual - Homens
homens.stat.corr("Age","Spending Score (1-100)",method="pearson")

Out[28]: -0.27962764627493125

In [36]:
#correlação entre Idade e o Salário Anual - Homens
mulheres.stat.corr("Age","Spending Score (1-100)",method="pearson")

Out[29]: -0.3754385837780473

Aplicando o Kmeans

In [38]:
from pyspark.ml.evaluation import ClusteringEvaluator  #biblioteca utilizada para a avaliação em cada um dos clusters
from pyspark.ml.clustering import KMeans #biblioteca utilizada para a criação do modelo de clusterização utilizando o K-means

In [39]:
from pyspark.ml.feature import VectorAssembler  #transformando os dados em vetores de características

vecAssembler = VectorAssembler(inputCols=["Spending Score (1-100)","Annual Income (k$)"], outputCol="features") #utilizada para transformar os dados em um vetor (define o objeto)
new_df = vecAssembler.transform(data) #Aplico a transformação
new_df.show()

+----------+------+---+------------------+----------------------+-----------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)| features|
+----------+------+---+------------------+----------------------+-----------+
 1| Male| 19| 15| 39|[39.0,15.0]|
 2| Male| 21| 15| 81|[81.0,15.0]|
 3|Female| 20| 16| 6| [6.0,16.0]|
 4|Female| 23| 16| 77|[77.0,16.0]|
 5|Female| 31| 17| 40|[40.0,17.0]|
 6|Female| 22| 17| 76|[76.0,17.0]|
 7|Female| 35| 18| 6| [6.0,18.0]|
 8|Female| 23| 18| 94|[94.0,18.0]|
 9| Male| 64| 19| 3| [3.0,19.0]|
 10|Female| 30| 19| 72|[72.0,19.0]|
 11| Male| 67| 19| 14|[14.0,19.0]|
 12|Female| 35| 19| 99|[99.0,19.0]|
 13|Female| 58| 20| 15|[15.0,20.0]|
 14|Female| 24| 20| 77|[77.0,20.0]|
 15| Male| 37| 20| 13|[13.0,20.0]|
 16| Male| 22| 20| 79|[79.0,20.0]|
 17|Female| 35| 21| 35|[35.0,21.0]|
 18| Male| 20| 21| 66|[66.0,21.0]|
 19| Male| 52| 23| 29|[29.0,23.0]|
 20|Female| 35| 23| 98|[98.0,23.0]|
+----------+------+---+------------------+----------------------+-----------+
only showing top 20 rows

In [40]:
#aplica o processo de clusterização
kmeans = KMeans(k=5, seed=1)  # declara o objeto - 3 clusters # Na verdade foi gerado 5 clusters
model = kmeans.fit(new_df.select('features')) #aplica o treinamento

In [41]:
#cria o dataset com a indicação sobre qual cluster cada conjunto de dados foi adicionado
df_final = model.transform(new_df)
df_final.show() 

+----------+------+---+------------------+----------------------+-----------+----------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)| features|prediction|
+----------+------+---+------------------+----------------------+-----------+----------+
 1| Male| 19| 15| 39|[39.0,15.0]| 4|
 2| Male| 21| 15| 81|[81.0,15.0]| 1|
 3|Female| 20| 16| 6| [6.0,16.0]| 4|
 4|Female| 23| 16| 77|[77.0,16.0]| 1|
 5|Female| 31| 17| 40|[40.0,17.0]| 4|
 6|Female| 22| 17| 76|[76.0,17.0]| 1|
 7|Female| 35| 18| 6| [6.0,18.0]| 4|
 8|Female| 23| 18| 94|[94.0,18.0]| 1|
 9| Male| 64| 19| 3| [3.0,19.0]| 4|
 10|Female| 30| 19| 72|[72.0,19.0]| 1|
 11| Male| 67| 19| 14|[14.0,19.0]| 4|
 12|Female| 35| 19| 99|[99.0,19.0]| 1|
 13|Female| 58| 20| 15|[15.0,20.0]| 4|
 14|Female| 24| 20| 77|[77.0,20.0]| 1|
 15| Male| 37| 20| 13|[13.0,20.0]| 4|
 16| Male| 22| 20| 79|[79.0,20.0]| 1|
 17|Female| 35| 21| 35|[35.0,21.0]| 4|
 18| Male| 20| 21| 66|[66.0,21.0]| 1|
 19| Male| 52| 23| 29|[29.0,23.0]| 4|
 20|Female| 35| 23| 98|[98.0,23.0]| 1|
+----------+------+---+------------------+----------------------+-----------+----------+
only showing top 20 rows

In [42]:
data.describe().show()

+-------+------------------+------+-----------------+------------------+----------------------+
summary| CustomerID| Genre| Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
 count| 200| 200| 200| 200| 200|
 mean| 100.5| null| 38.85| 60.56| 50.2|
 stddev|57.879184513951124| null|13.96900733155888| 26.26472116527124| 25.823521668370173|
 min| 1|Female| 18| 15| 1|
 max| 200| Male| 70| 137| 99|
+-------+------------------+------+-----------------+------------------+----------------------+

In [43]:
df_final.groupby('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 22|
 3| 35|
 4| 23|
 2| 39|
 0| 81|
+----------+-----+

In [44]:
#transformando os dados utilizando o pandas 
data_pandas = df_final.toPandas().set_index('CustomerID')
data_pandas.head()

,Genre,Age,Annual Income (k$),Spending Score (1-100),features,prediction
CustomerID,,,,,,
1,Male,19,15,39,"[39.0, 15.0]",4
2,Male,21,15,81,"[81.0, 15.0]",1
3,Female,20,16,6,"[6.0, 16.0]",4
4,Female,23,16,77,"[77.0, 16.0]",1
5,Female,31,17,40,"[40.0, 17.0]",4


In [45]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))
plt.scatter(data_pandas['Genre'], data_pandas['Age'], c=data_pandas.prediction)
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.show()

In [46]:
new_df2 = vecAssembler.transform(data) #Aplico a transformação
new_df2.show()

+----------+------+---+------------------+----------------------+-----------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)| features|
+----------+------+---+------------------+----------------------+-----------+
 1| Male| 19| 15| 39|[39.0,15.0]|
 2| Male| 21| 15| 81|[81.0,15.0]|
 3|Female| 20| 16| 6| [6.0,16.0]|
 4|Female| 23| 16| 77|[77.0,16.0]|
 5|Female| 31| 17| 40|[40.0,17.0]|
 6|Female| 22| 17| 76|[76.0,17.0]|
 7|Female| 35| 18| 6| [6.0,18.0]|
 8|Female| 23| 18| 94|[94.0,18.0]|
 9| Male| 64| 19| 3| [3.0,19.0]|
 10|Female| 30| 19| 72|[72.0,19.0]|
 11| Male| 67| 19| 14|[14.0,19.0]|
 12|Female| 35| 19| 99|[99.0,19.0]|
 13|Female| 58| 20| 15|[15.0,20.0]|
 14|Female| 24| 20| 77|[77.0,20.0]|
 15| Male| 37| 20| 13|[13.0,20.0]|
 16| Male| 22| 20| 79|[79.0,20.0]|
 17|Female| 35| 21| 35|[35.0,21.0]|
 18| Male| 20| 21| 66|[66.0,21.0]|
 19| Male| 52| 23| 29|[29.0,23.0]|
 20|Female| 35| 23| 98|[98.0,23.0]|
+----------+------+---+------------------+----------------------+-----------+
only showing top 20 rows

In [47]:
#aplica o processo de clusterização
kmeans2 = KMeans(k=3, seed=1)  # declara o objeto - 3 clusters # Na verdade foi gerado 5 clusters
model2 = kmeans2.fit(new_df2.select('features')) #aplica o treinamento

In [48]:
df_final2 = model.transform(new_df2)
df_final2.show() 

+----------+------+---+------------------+----------------------+-----------+----------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)| features|prediction|
+----------+------+---+------------------+----------------------+-----------+----------+
 1| Male| 19| 15| 39|[39.0,15.0]| 4|
 2| Male| 21| 15| 81|[81.0,15.0]| 1|
 3|Female| 20| 16| 6| [6.0,16.0]| 4|
 4|Female| 23| 16| 77|[77.0,16.0]| 1|
 5|Female| 31| 17| 40|[40.0,17.0]| 4|
 6|Female| 22| 17| 76|[76.0,17.0]| 1|
 7|Female| 35| 18| 6| [6.0,18.0]| 4|
 8|Female| 23| 18| 94|[94.0,18.0]| 1|
 9| Male| 64| 19| 3| [3.0,19.0]| 4|
 10|Female| 30| 19| 72|[72.0,19.0]| 1|
 11| Male| 67| 19| 14|[14.0,19.0]| 4|
 12|Female| 35| 19| 99|[99.0,19.0]| 1|
 13|Female| 58| 20| 15|[15.0,20.0]| 4|
 14|Female| 24| 20| 77|[77.0,20.0]| 1|
 15| Male| 37| 20| 13|[13.0,20.0]| 4|
 16| Male| 22| 20| 79|[79.0,20.0]| 1|
 17|Female| 35| 21| 35|[35.0,21.0]| 4|
 18| Male| 20| 21| 66|[66.0,21.0]| 1|
 19| Male| 52| 23| 29|[29.0,23.0]| 4|
 20|Female| 35| 23| 98|[98.0,23.0]| 1|
+----------+------+---+------------------+----------------------+-----------+----------+
only showing top 20 rows

In [49]:
data.describe().show()

+-------+------------------+------+-----------------+------------------+----------------------+
summary| CustomerID| Genre| Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
 count| 200| 200| 200| 200| 200|
 mean| 100.5| null| 38.85| 60.56| 50.2|
 stddev|57.879184513951124| null|13.96900733155888| 26.26472116527124| 25.823521668370173|
 min| 1|Female| 18| 15| 1|
 max| 200| Male| 70| 137| 99|
+-------+------------------+------+-----------------+------------------+----------------------+

In [50]:
df_final2.groupby('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 22|
 3| 35|
 4| 23|
 2| 39|
 0| 81|
+----------+-----+

In [51]:
#transformando os dados utilizando o pandas 
data_pandas2 = df_final2.toPandas().set_index('CustomerID')
data_pandas2.head()

,Genre,Age,Annual Income (k$),Spending Score (1-100),features,prediction
CustomerID,,,,,,
1,Male,19,15,39,"[39.0, 15.0]",4
2,Male,21,15,81,"[81.0, 15.0]",1
3,Female,20,16,6,"[6.0, 16.0]",4
4,Female,23,16,77,"[77.0, 16.0]",1
5,Female,31,17,40,"[40.0, 17.0]",4


In [52]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))
plt.scatter(data_pandas2['Genre'], data_pandas2['Age'], c=data_pandas2.prediction)
plt.xlabel('Male')
plt.ylabel('Age')
plt.show()

In [53]:
from pyspark.sql.functions import mean, stddev

llista = data.select(mean(col('Age')).alias('media'),stddev(col('Age')).alias('desvioPadrao'),mean(col('Annual Income (k$)'))
                     .alias('media k$'),stddev(col('Annual Income (k$)')).alias('desvioPadrao k$')).collect()
media = llista[0]['media']
mediakS = llista[0]['media k$']

desvio = llista[0]['desvioPadrao']
desviokS = llista[0]['desvioPadrao k$']

print("Média: ", media)
print("Média K$: ", mediakS)

print("Desvio Padrão: ", desvio)
print("Desvio Padrão K$: ", desviokS)

Média: 38.85
Média K$: 60.56
Desvio Padrão: 13.96900733155888
Desvio Padrão K$: 26.26472116527124

In [54]:
display(data.select(["Genre","Annual Income (k$)", "Age"]))

Genre,Annual Income (k$),Age
Male,15,19
Male,15,21
Female,16,20
Female,16,23
Female,17,31
Female,17,22
Female,18,35
Female,18,23
Male,19,64
Female,19,30


In [55]:
homem = data[data['Genre'] == 'Male']
homem.describe().show()

+-------+------------------+-----+------------------+------------------+----------------------+
summary| CustomerID|Genre| Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+-----+------------------+------------------+----------------------+
 count| 88| 88| 88| 88| 88|
 mean|104.23863636363636| null| 39.80681818181818| 62.22727272727273| 48.51136363636363|
 stddev| 57.48382967392823| null|15.514811576858186|26.638373182494135| 27.896769605833605|
 min| 1| Male| 18| 15| 1|
 max| 200| Male| 70| 137| 97|
+-------+------------------+-----+------------------+------------------+----------------------+

In [56]:
mulher = data[data['Genre'] == 'Female']
mulher.describe().show()

+-------+-----------------+------+------------------+------------------+----------------------+
summary| CustomerID| Genre| Age|Annual Income (k$)|Spending Score (1-100)|
+-------+-----------------+------+------------------+------------------+----------------------+
 count| 112| 112| 112| 112| 112|
 mean| 97.5625| null|38.098214285714285| 59.25| 51.526785714285715|
 stddev|58.27641206099346| null|12.644095457392353|26.011951515055948| 24.114949877478647|
 min| 3|Female| 18| 16| 5|
 max| 197|Female| 68| 126| 99|
+-------+-----------------+------+------------------+------------------+----------------------+